In [1]:
import numpy as np
import pandas as pd

In [11]:
from sentence_transformers import SentenceTransformer
from torch.nn import CosineSimilarity
import torch
import pickle
import json

In [12]:
json_path = "faq.json"
with open(json_path, "r") as f:
    training_data = json.load(f)

In [15]:
import pandas as pd
#Reading the JSON File
df = pd.read_json("faq.json")
#Printing the data Frame
df.head()

,question,answer
0,Why is my HVAC system consuming too much power?,High power consumption is likely due to an inc...
1,What could cause an HVAC unit to have low cool...,Low cooling efficiency may be indicated by a s...
2,Why is there an error in my HVAC system?,An error is detected based on system fault cod...
3,Why is the temperature difference lower than e...,A lower-than-expected temperature difference m...
4,What does a sudden increase in flow rate indic...,A sudden spike in airflow may indicate a valve...


In [17]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [18]:
MODEL_PATH = r"model_va.pickle"

In [19]:

q_embs = model.encode(df["question"]) # computes encode for all the questions from the dataset.
                                    #Embeddings can be computed in batches for massive dataset.


In [20]:
with open(MODEL_PATH, "wb") as file:
    pickle.dump(q_embs, file)

In [21]:
with open(r"model_va.pickle", "rb") as file:
    q_embs = pickle.load(file)

In [24]:
def pred_answer(usr_query):
    df_query = pd.DataFrame([usr_query], columns=["usr_query"]) # use similar pipeline that was used for computing embeddings from dataset
    usr_q_emb = model.encode(df_query["usr_query"]) # compute embedding
    cosine_similarity = CosineSimilarity()
    q_idx = np.argmax(cosine_similarity(torch.from_numpy(usr_q_emb), torch.from_numpy(q_embs))) # compute cosine similarity and find the matched query
    return (df["answer"][q_idx.item()]) # look up answer of the matched query from the dataframe of inp

In [28]:
while True:
    usr_query = input("Ask a query:")
    if usr_query == "exit":
        break
    else:
        print("Answer: ", pred_answer(usr_query))
    print("-----------------")

Ask a query:exit
